In [2]:
################################################
# Funtion to create directories (if not exist)
################################################

def create_directory(dir_to_create_path):
  dirExist = os.path.exists(dir_to_create_path)

  if not dirExist:
    os.makedirs(dir_to_create_path)
    #print("The new directory for %s image is created!" % (slide_num))
    return (dir_to_create_path)

In [ ]:
################################################
# Function to read the slides
################################################

def read_slide(slide, x, y, level, width, height, as_float=False):
    im = slide.read_region((x,y), level, (width, height))
    im = im.convert('RGB') # drop the alpha channel
    if as_float: 
        im = np.asarray(im, dtype=np.float32)
    else:
        im = np.asarray(im)
    assert im.shape == (height, width, 3)
    return im

In [ ]:
################################################
# Function to remove BlackGround
################################################

def backgroundRemoval(slide_image):
    mask = grays_filter(slide_image) # tcheck here if everthing is blavk, so just f
    img_background_removed = slide_image * np.dstack([mask, mask, mask])
    return img_background_removed

def grays_filter(rgb, tolerance=15):
  
    #Mask to filter out pixels where the red, green, and blue channel values are similar.
    '''
    Filtro para a área de sombra no slide (área de sombra = gradiente de tons de cinza escuro a claro). 
    Um pixel cinza tem valores de canal vermelho, verde e azul que estão próximos. 
    A função filter_grays () filtra os pixels que têm valores de vermelho, azul e verde que estão dentro de uma certa tolerância um do outro. 
    A tolerância padrão é 15. O filtro cinzas também filtra pixels brancos e pretos, pois eles têm valores semelhantes de vermelho, verde e azul.
    '''
    rgb = rgb.astype(int)
    rg_diff = abs(rgb[:, :, 0] - rgb[:, :, 1]) <= tolerance
    rb_diff = abs(rgb[:, :, 0] - rgb[:, :, 2]) <= tolerance
    gb_diff = abs(rgb[:, :, 1] - rgb[:, :, 2]) <= tolerance
    result = ~(rg_diff & rb_diff & gb_diff)

    return result

In [ ]:
#####################################################################
# Prepare image to be saved as rgb image - Filter with no tolerance
#####################################################################

def img_to_rgb(rgb, tolerance=0):
  
    rgb = rgb.astype(int)
    rg_diff = abs(rgb[:, :, 0] - rgb[:, :, 1]) <= tolerance
    rb_diff = abs(rgb[:, :, 0] - rgb[:, :, 2]) <= tolerance
    gb_diff = abs(rgb[:, :, 1] - rgb[:, :, 2]) <= tolerance
    result = ~(rg_diff & rb_diff & gb_diff)

    return result

In [ ]:
#####################################################################
# Check if there is positive pixel in the center patch of the image
#####################################################################

def isCenterMaskPositive(mask):
   qty_positive_pixel = np.count_nonzero(mask)
   if(qty_positive_pixel >= 3):
     return True
   else: 
      return False

In [ ]:
#####################################################################
# Check if patch is has just positive pixels
#####################################################################

#150528 = (224*224*3) minus 528 as margin of 14px
#149500 = (224*224*3) minus 1028 as margin of 28px

def isPatchNotFullPositive(patch):
   qty_positive_pixel = np.count_nonzero(patch)
   if(qty_positive_pixel < 149500):
     return True
   else: 
      return False

In [ ]:
#####################################################################
# Convert an HxWx3 numpy array into an RGB Image
#####################################################################

def numpy2RGBpil(np_array: np.ndarray) -> Image:

    assert_msg = 'Input shall be a HxWx3 ndarray'
    assert isinstance(np_array, np.ndarray), assert_msg
    assert len(np_array.shape) == 3, assert_msg
    assert np_array.shape[2] == 3, assert_msg

    img = Image.fromarray(np_array, 'RGB')
    return img

In [1]:
# Function to normalize stain
def normalizeStaining(img, saveFile=None, Io=240, alpha=1, beta=0.15):
             
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
        
    maxCRef = np.array([1.9705, 1.0308])
    
    # define height and width of image
    h, w, c = img.shape
    
    # reshape image
    img = img.reshape((-1,3))

    # calculate optical density
    OD = -np.log((img.astype(float)+1)/Io)
    
    # remove transparent pixels
    ODhat = OD[~np.any(OD<beta, axis=1)]
        
    # compute eigenvectors
    eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))
        
    # project on the plane spanned by the eigenvectors corresponding to the two 
    # largest eigenvalues    
    That = ODhat.dot(eigvecs[:,1:3])
    
    phi = np.arctan2(That[:,1],That[:,0])
    
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100-alpha)
    
    vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)
    
    # a heuristic to make the vector corresponding to hematoxylin first and the 
    # one corresponding to eosin second
    if vMin[0] > vMax[0]:
        HE = np.array((vMin[:,0], vMax[:,0])).T
    else:
        HE = np.array((vMax[:,0], vMin[:,0])).T
    
    # rows correspond to channels (RGB), columns to OD values
    Y = np.reshape(OD, (-1, 3)).T
    
    # determine concentrations of the individual stains
    C = np.linalg.lstsq(HE,Y, rcond=None)[0]
    
    # normalize stain concentrations
    maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
    tmp = np.divide(maxC,maxCRef)
    C2 = np.divide(C,tmp[:, np.newaxis])
    
    # recreate the image using reference mixing matrix
    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm>255] = 254
    plt.figure(figsize=(10,10), dpi=100)
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)  
        
    #return Inorm
    return Image.fromarray(Inorm)

In [11]:
def imageHasTissue(image, tolerance):
    qty_tissue_pixel = np.count_nonzero(image)
    if(qty_tissue_pixel > tolerance):
        return True
    else: 
        return False

In [4]:
def percentTissueAtImage(image, itemSizeInPixels):
    qtyTissuePixel = np.count_nonzero(image)
    channel = 3 #RGB
    percentTissuePixel = ((qtyTissuePixel/channel)/(itemSizeInPixels*itemSizeInPixels))*100 
    
    return percentTissuePixel

In [12]:
####################################################
# Cosmetic Print Informations
####################################################

def print_slide_info(slideWidth, slideHeight, numPatchesX, numPatchesY, slide_path, all_patches_dir):
    print('slideWidth =', slideWidth)
    print('slideHeight =', slideHeight)
    print('num lin =', numPatchesX)
    print('num col =', numPatchesY)
    print('total patches =', numPatchesX*numPatchesY)
    print('----------------------------------------')
    print(slide_path)
    print(all_patches_dir)

def print_crop_info(patch_name, numOfPatches, numOfExtractedPatches, numOfNormedPatches, numOfNonNormedPatches):
    print('Last patch_name', patch_name)
    print('numOfPatches=', numOfPatches)
    print('numOfExtractedPatches=', numOfExtractedPatches)
    print('numOfNormedPatches=', numOfNormedPatches)
    print('numOfNonNormedPatches=', numOfNonNormedPatches)


def print_crop_info_without_norm(patch_name, numOfPatches, numOfExtractedPatches, numOfExceptions):
    print('Last patch_name', patch_name)
    print('numOfPatches=', numOfPatches)
    print('numOfExtractedPatches=', numOfExtractedPatches)
    print('numOfExceptions=', numOfExceptions)